# Optimization in Python/Jupyter

## Table of Contents
1. [Introduction](#introduction)
2. [Profit Maximization (crop mix)](#crop_mix)
3. [Profit Maximization (production)](#production)
4. [Cost Minimization (supply cost)](#supply_cost)
5. [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

Python has a variety of libraries available for assiting an analyst with optimization. In this notebook we will explore linear programming with the pyomo library and the glpk solver. 

## Profit Maximization (crop mix) <a name="crop_mix"></a>
Problem Source: Mathematical Applications For The Management, Life and Social Sciences (eighth edition, authors Harshbarger and Reynolds) - page 293. The solution was not given in python. The book uses a graphical method and the simplex method to solve. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyomo.environ as pe
import pyomo.opt as po
import pandas as pd

Given: <br>
1. 6000 acres of land available
2. Each acre of corn requires 9 gallons of fertilizer
3. Each acre of soybeans requires 3 gallons of fertilizer
4. Each acre of corn requires 0.75 hours of labor to harvest
5. Each acre of soybeans requires 1 hour of labor to harvest
6. There is at most 40500 gallons of fertilizer available
7. There is at most 5250 hours of labor available
8. The profit for corn is \$60 per acre 
9. The profit for soybeans is \$40 per acre

From given we get the following system of inequalities:
1. $$x + y \le 6000$$
2. $$9x + 3y \le 40500$$
3. $$0.75x + y \le 5250$$

Profit Function:
$$P(x, y) = 60x + 40y$$

In [2]:
model = pe.ConcreteModel()

#declare decision variables
model.x = pe.Var(domain = pe.NonNegativeReals)
model.y = pe.Var(domain = pe.NonNegativeReals)

#declare objective
model.profit = pe.Objective(expr = 60 * model.x + 40 * model.y, sense = pe.maximize)

#declare constraints
model.acres = pe.Constraint(expr = model.x + model.y <= 6000)
model.fertilizer = pe.Constraint(expr = 9 * model.x + 3 * model.y <= 40500)
model.hours = pe.Constraint(expr = 0.75 * model.x + model.y <= 5250)

#solve
opt = po.SolverFactory('glpk')
opt.solve(model);

In [3]:
print("Maximized profit = ", model.profit())
print("Optimized acres of corn = ", model.x())
print("Optimized acres of soybeans = ", model.y())

Maximized profit =  315000.0
Optimized acres of corn =  3750.0
Optimized acres of soybeans =  2250.0


## Profit Maxization (production) <a name="production"></a>
Problem Source: Mathematical Applications For The Management, Life and Social Sciences (eighth edition, authors Harshbarger and Reynolds) - page 345 (#29). The solution was not given in python. The book uses the simplex method to solve. 

Given:
1. Each pound of all-beef hot dogs requires 0.75 lb of beef and 0.2 lb of spices.
2. Each pound of regular hot dogs requires 0.18 lb of beef and 0.3 lbs of pork and 0.2 lb of spices
3. Suppliers can deliver at most 1020 lb of beef
4. Suppliers can deliver at most 600 lb of pork
5. Suppliers can deliver at least 500 lb of spices
6. Profit is \$0.6 on each pound of all-beef hot dogs
7. Profit is \$0.4 on each pound of regular hot dogs

From given we get the following system of inequalities:
1. $$0.75x + 0.18y \le 1020$$
2. $$0.2x + 0.2y \ge 500$$
3. $$0.3y \le 600$$

Profit Function:
$$P(x,y) = 0.6x + 0.4y$$

In [4]:
model = pe.ConcreteModel()

#declare decision variables
model.x = pe.Var(domain = pe.NonNegativeReals)
model.y = pe.Var(domain = pe.NonNegativeReals)

#declare objective
model.profit = pe.Objective(expr = 0.6 * model.x + 0.4 * model.y, sense = pe.maximize)

#declare constraints
model.beef = pe.Constraint(expr = 0.75 * model.x + 0.18 * model.y <= 1020)
model.spices = pe.Constraint(expr = 0.2 * model.x + 0.2 * model.y >= 500)
model.pork = pe.Constraint(expr = 0.3 * model.y <= 600)

#solve
opt = po.SolverFactory('glpk')
opt.solve(model);

In [5]:
print("Maximized profit = ", model.profit())
print("Optimized pounds of beef produced = ", model.x())
print("Optimized pounds of regular produced = ", model.y())

Maximized profit =  1328.0
Optimized pounds of beef produced =  880.0
Optimized pounds of regular produced =  2000.0


## Cost Minimization (supply cost)<a name="supply_cost"></a>
Problem Source: Mathematical Applications For The Management, Life and Social Sciences (eighth edition, authors Harshbarger and Reynolds) - page 336 (#31). The solution was not given in python. The book uses the simplex method to solve. 

Given:
1. Need at least 230 reams of recycled paper
2. Need at least 240 reams of satin white paper
3. Need at least 210 reams of glossy white paper
4. Addition information:

In [6]:
details = {
    'Recycled':[10, 5, 6],
    'Satin White':[5, 8, 6],
    'Glossy White': [6, 6, 6],
    'Package Cost': [120, 140, 126]
}
data = pd.DataFrame(data = details, index = ['Georgia Package', 'Union Package', 'Pacific Package'])
data

,Recycled,Satin White,Glossy White,Package Cost
Georgia Package,10,5,6,120
Union Package,5,8,6,140
Pacific Package,6,6,6,126


From given we get the following system of inequalities:
1. $$10x + 5y + 6z \ge 230$$
2. $$5x + 8y + 6z \ge 240$$
3. $$6x + 6y + 6z \ge 210$$

Cost function:
$$C(x, y, z) = 120x + 140y + 126z$$

In [7]:
model = pe.ConcreteModel()

#declare decision variables
model.x = pe.Var(domain = pe.NonNegativeReals)
model.y = pe.Var(domain = pe.NonNegativeReals)
model.z = pe.Var(domain = pe.NonNegativeReals)

#declare objective
model.cost = pe.Objective(expr = 120 * model.x + 140 * model.y + 126 * model.z, sense = pe.minimize)

#declare constraints
model.output_a = pe.Constraint(expr = 10 * model.x + 5 * model.y + 6 * model.z >= 230)
model.output_b = pe.Constraint(expr = 5 * model.x + 8 * model.y + 6 * model.z >= 240)
model.restrict = pe.Constraint(expr = 6 * model.x + 6 * model.y + 6 * model.z >= 210)

#solve
opt = po.SolverFactory('glpk')
opt.solve(model);

In [8]:
print("Minimized cost = ", model.cost())
print("Optimized reams of recycled purchased = ", model.x())
print("Optimized reams of satin white purchased = ", model.y())
print("Optimized reams of glossy white purchased = ", model.z())

Minimized cost =  4630.0
Optimized reams of recycled purchased =  10.0
Optimized reams of satin white purchased =  20.0
Optimized reams of glossy white purchased =  5.0


# Conclusion <a name="conclusion"></a>
Python has a wide variety of useful libraries available such as pyomo to help solve a variety of optimization problems. 